<a href="https://colab.research.google.com/github/kennycaiguo/-LearnPytorchWithColab/blob/main/torch_tutorial_lesson2_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self,x):
      x = F.relu(self.fc1(x))
      x = self.fc2(x)
      return x

# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(device)
#超参数设置
input_size = 784
num_classes = 10
learnng_rate = 0.001

batch_size = 64
num_epochs = 1

# 加载数据集
train_dataset = datasets.MNIST(root="dataset/",train=True,transform=transforms.ToTensor(),download=True)
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

test_dataset = datasets.MNIST(root="dataset/",train=False,transform=transforms.ToTensor(),download=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)
# 初始化神经网络
model = NN(input_size=input_size,num_classes=num_classes).to(device)

#损失函数和优化器
criteron = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learnng_rate)

#训练模型
for epoch in range(num_epochs):
  for batch_idx,(data,targets) in enumerate(train_loader):
    # 把数据发送到cuda(如果可用)
    data=data.to(device=device)
    targets=targets.to(device=device)
    # print(data.shape)
    # get the correct shape
    data = data.reshape(data.shape[0],-1)
    # forward
    scores = model(data)
    # loss
    loss = criteron(scores,targets)
    # 反向传播
    optimizer.zero_grad() # 清空优化器的梯度缓存
    loss.backward()

    #梯度下降炒操作
    optimizer.step()

#模型验证,检验准确率
def check_accuracy(loader,model):
  if loader.dataset.train:
    print("Checking accuracy on Train data")
  else:
    print("Checking accuracy on Test data")
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x,y in loader:
      x = x.to(device=device)
      y = y.to(device=device)
      x = x.reshape(x.shape[0],-1)

      scores = model(x)
      _,predictions = scores.max(1)
      num_correct += (predictions==y).sum()
      num_samples += predictions.size(0)
    print(f"got {num_correct}/{num_samples} with accuracy:{float(num_correct)/float(num_samples)*100:.2f}")

  model.train()
  return num_correct / num_samples

check_accuracy(train_loader,model)
check_accuracy(test_loader,model)





Checking accuracy on Train data
got 55992/60000 with accuracy:93.32
Checking accuracy on Test data
got 9312/10000 with accuracy:93.12


tensor(0.9312, device='cuda:0')